In [70]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score
from sklearn.cluster import KMeans

Identifying birds that strongly select for one or more habitats

In [71]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [72]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [110]:
##TODO #99 replace the update latestUpdate in the function with a trigger in the database
class notInTable(ValueError):
    def __init__(self,locId,message="given locId cannot be found in table Hotspots"):
        self.locId = locId
        self.message = message
        super().__init__(self.message)
    def __str__(self):
        return f'{self.locId} >> {self.message}'



In [146]:

def post_hotspotHabitat(locId: str,label: int,cnx=connectDB()):
    try:
        cursor = cnx.cursor()
    #validate
        validate_query = "SELECT EXISTS(SELECT ? FROM Hotspots WHERE locId = ?)"
        validate_tuple = (locId,locId)
        cursor.execute(validate_query,validate_tuple)
        validate = cursor.fetchone()[0]
        if validate == 0:
            raise notInTable(locId)
        else:
            updateLabel_script = "UPDATE Hotspots SET habitatLabel = ?, latestUpdate = ? WHERE locId = ?;"
            updatelabel_tuple = (label,str(dt.datetime.today()),locId)
            cursor.execute(updateLabel_script,updatelabel_tuple)
            cnx.commit()
            cursor.close()
    except ValueError as vxf:
        raise vxf
    except Exception as ex:
        raise ex
    return locId

In [83]:
#derive habitat cluster labels from FAO values
def kmeans_habitat(distinctHabitats: int,cnx= connectDB()):
    try:
        data = pd.read_sql(sql='SELECT * FROM FAO_by_locId;',con=cnx)
        data = data.drop(columns=['locName']).set_index('locId')
        data.fillna(0,inplace=True)
    #normalize
        maxValue = data.apply(max,axis=1)
        data = data.apply(lambda x: (x/maxValue[x.index]),axis=0)   #min-max normalizing to smooth in proportionality
    #compute kmeans for each locId
        habitat_kmeans = KMeans(n_clusters=distinctHabitats,init='k-means++')
        habitat_kmeans = habitat_kmeans.fit(data.values)
        clusterLabels = habitat_kmeans.labels_
        centerPoints = habitat_kmeans.cluster_centers_
    #define habitats
        habitatFrame = pd.DataFrame(data=clusterLabels,columns=['clusterLabel'],index=data.index).sort_values(by='clusterLabel').reset_index()
        #habitatFrame = pd.merge(left=habitatFrame,left_on='locId',right=data,right_on='locId',how='left')
    except Exception as kmeansExc:
        raise kmeansExc
    return habitatFrame,centerPoints

Test and interpolate

In [122]:
##call function into frame
query = f"""with habitatAbundance as (
select
	(strftime('%Y',obsDt)||'-'||strftime('%W',obsDt)) as "year_week",
	round(avg(howMany),0) as "habitatVolume",
	(total(round(avg(howMany),0)) over(partition by strftime('%Y',obsDt)||'-'||strftime('%W',obsDt))) as "volume_allHabitats",
	habitatLabel 
from historicObservations as hsob
left join (select locId,habitatLabel from Hotspots) as "labels" on hsob.locId=labels.locId
where speciesCode = '{speciesCode}'
group by year_week,habitatLabel
)
select year_week,round((habitatVolume/volume_allHabitats),2) as "selectionRatio_{i}"
from habitatAbundance
where habitatLabel = {i}
;"""

In [156]:
def build_datebase(speciesCode,con = connectDB()):
    try:
        cursor = con.cursor()
        minYearQuery = f"""select strftime('%Y',obsDt) as "year" from historicObservations group by speciesCode having obsDt=min(obsDt) and speciesCode = '{speciesCode}';"""
        maxYearQuery = f"""select strftime('%Y',obsDt) as "year" from historicObservations group by speciesCode having obsDt=max(obsDt) and speciesCode = '{speciesCode}';"""
        minYear = cursor.execute(minYearQuery).fetchone()[0]
        maxYear = cursor.execute(maxYearQuery).fetchone()[0]
        datebase = []
        for x in range(int(minYear),int(maxYear),1):
            for y in range(1,53,1):
                week_year = '-'.join([str(x),str(y)])
                datebase.append(week_year)
    except Exception as ex:
        raise ex
    return datebase


In [157]:
build_datebase(speciesCode='westan')

['2019-1',
 '2019-2',
 '2019-3',
 '2019-4',
 '2019-5',
 '2019-6',
 '2019-7',
 '2019-8',
 '2019-9',
 '2019-10',
 '2019-11',
 '2019-12',
 '2019-13',
 '2019-14',
 '2019-15',
 '2019-16',
 '2019-17',
 '2019-18',
 '2019-19',
 '2019-20',
 '2019-21',
 '2019-22',
 '2019-23',
 '2019-24',
 '2019-25',
 '2019-26',
 '2019-27',
 '2019-28',
 '2019-29',
 '2019-30',
 '2019-31',
 '2019-32',
 '2019-33',
 '2019-34',
 '2019-35',
 '2019-36',
 '2019-37',
 '2019-38',
 '2019-39',
 '2019-40',
 '2019-41',
 '2019-42',
 '2019-43',
 '2019-44',
 '2019-45',
 '2019-46',
 '2019-47',
 '2019-48',
 '2019-49',
 '2019-50',
 '2019-51',
 '2019-52',
 '2020-1',
 '2020-2',
 '2020-3',
 '2020-4',
 '2020-5',
 '2020-6',
 '2020-7',
 '2020-8',
 '2020-9',
 '2020-10',
 '2020-11',
 '2020-12',
 '2020-13',
 '2020-14',
 '2020-15',
 '2020-16',
 '2020-17',
 '2020-18',
 '2020-19',
 '2020-20',
 '2020-21',
 '2020-22',
 '2020-23',
 '2020-24',
 '2020-25',
 '2020-26',
 '2020-27',
 '2020-28',
 '2020-29',
 '2020-30',
 '2020-31',
 '2020-32',
 '2020-33'